In [1]:
Threads.nthreads()

1

In [2]:
using Gridap
import Gridap: ∇
using LinearAlgebra
using GridapGmsh

In [3]:
#Mallado
model = GmshDiscreteModel("test-gridap/circle.msh")
const global degree = 1
Ω = Triangulation(model)
dΩ = Measure(Ω,2*degree)
quad = CellQuadrature(Ω,degree)

Info    : Reading 'test-gridap/circle.msh'...
Info    : 423 nodes
Info    : 844 elements
Info    : Done reading 'test-gridap/circle.msh'


CellQuadrature()

In [4]:
#Parámetros

u_input(x) = 0.  #valor en la frontera izquierda
betaf(x) = VectorValue(-x[2],x[1]) #vector de transporte
u_init(x) = exp(-1000*((x[1]-0.3)^2+(x[2])^2)) #Condición inicial

#Partición en tiempo
const global t_init = 0.
t_end = 2*π

N_intervalos = 100;

n_iter = 1;
T = LinRange(t_init, t_end, N_intervalos+1);

In [5]:
#Espacio de funciones
order = 1
reffe = ReferenceFE(lagrangian,Float64,order)
V0 = TestFESpace(model,reffe,dirichlet_tags=["Dirichlet"])

U = TrialFESpace(V0)

#beta = interpolate_everywhere(betaf,U)
#beta = get_free_dof_values(beta)

UnconstrainedFESpace()

In [6]:
function MassLumping(M)
    n = size(M)[1]
    ML = zeros(n,n)
    for i = 1:n
        ML[i,i] = sum(M[i,:])
        end
    return ML
end

MassLumping (generic function with 1 method)

In [7]:
function AddDifussion(C)
    n = size(C)[1]
    L = zeros(n,n)
    for i=1:n
        d_diag = 0
        for j =1:(i-1)
            d = max(-C[i,j],0, -C[j,i])
            L[i,j]= C[i,j] + d
            L[j,i]= C[j,i] + d
            d_diag += d
        end
        L[i,i] = C[i,i] - d_diag
    end
    return L
end

AddDifussion (generic function with 1 method)

In [8]:
function DifussionArtificial(C)
    n = size(C)[1]
    D = zeros(n,n)
    for i=1:n
        for j=1:(i-1)
            D[i,j] = D[j,i] = max(-C[i,j],0,-C[j,i])
            end
        end 
    for i=1:n
        D[i,i] = -sum(D[i,:])
        end 
    return D
end 

DifussionArtificial (generic function with 1 method)

In [9]:
function TestPositividad(A,B)
    n = size(A)[1]
    for i=1:n
        if(A[i,i]<=0)
            throw(error("Elemento diagonal negativo"))
        end
        for j=1:(i-1)
            if(A[i,j]>0)
            throw(error("Elemento no diagonal positivo"))
        end
        end 
        for j=(i+1):n
            if(A[i,j]>0)
            throw(error("Elemento no diagonal positivo"))
        end
        end 
        if(sum(A[i,:])<0)
            throw(error("Fila con suma negativa"))
        end 
        if(B[i]<0)
            throw(error("Elemento de B negativo"))
        end 
    end 
    
end 
        

TestPositividad (generic function with 1 method)

In [10]:
FE_function_type() = Gridap.CellData.OperationCellField

FE_function_type (generic function with 1 method)

In [25]:
function EulerMEF(t_init,t_end,n_t,u_init)
    dt = (t_end - t_init)/n_t
    t = t_init
    u0 = interpolate_everywhere(u_init,U)
    u0 = get_free_dof_values(u0)
    
    a_convec(u,v) = ∫(betaf⋅∇(u)*v)*dΩ #conveccion
    a_masa(u,v)= ∫(v*u)*dΩ  #matriz masa
    
    uf = get_trial_fe_basis(V0)
    vf = get_fe_basis(V0)
    cell_dofs = get_cell_dof_ids(V0)
    assem = SparseMatrixAssembler(V0,V0)
    
    cell_mat_masa = a_masa(uf,vf)[Ω]
    data_masa = ([cell_mat_masa], [cell_dofs], [cell_dofs])
    M = assemble_matrix(assem,data_masa)
    ML = MassLumping(M)
    
    cell_mat_convec = a_convec(uf,vf)[Ω]
    data_convec = ([cell_mat_convec], [cell_dofs], [cell_dofs])
    K = assemble_matrix(assem,data_convec)
    
    D = DifussionArtificial(-K)
    
    L = -K + D
    
    A = ML - dt*L
    B = ML*u0  # multiplicar matriz por el vector u0
    
    #TestPositividad(A,B)

    for i=1:n_t
        t += dt
        u_sol = A\B
        u0 = FEFunction(U,u_sol)
        
        B = ML*u_sol  # multiplicar matriz por el vector u0
    end
    return u0
end 

EulerMEF (generic function with 1 method)

In [26]:
EulerMEF(t_init,t_end,N_intervalos,u_init)

SingleFieldFEFunction():
 num_cells: 780
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 606179847501829834

In [27]:
t_n_f = 20;
t_n_c = 10;

F(t1, t0, u0) =  EulerMEF(t0, t1, t_n_f,u0)
G(t1, t0, u0) =  EulerMEF(t0, t1, t_n_c,u0)

G (generic function with 1 method)

In [28]:
dt = (t_end - t_init)/N_intervalos

u0_guardar = EulerMEF(t_init, t_init+dt, 1, u_init)

typeof(u0_guardar)
FE_function_type() = Gridap.CellData.OperationCellField

FE_function_type (generic function with 1 method)

In [29]:
function EDP_secuencial()
    U = Array{FE_function_type(),2}(undef, N_intervalos+1, n_iter+1);
    F_sol = Array{FE_function_type(),1}(undef, N_intervalos+1);


    # 1.a) Inicialización (aproximción grosera)
    U[1,1] = 1*u0_guardar
    

    for n=1:N_intervalos
        U[n+1,1] = 1*G( T[n+1],T[n],U[n,1] )
    end
    
        
    # 1.b) Inicialización etapas parareal
    for k=1:n_iter
        U[1,k+1] = 1*u0_guardar
    end

    # 2) Bucle parareal
    for k=1:n_iter
 
        # 2.a) Aproximación fina (paralela) en cada subintervalo
        for n = 1:N_intervalos
            F_sol[n] = 1*F( T[n+1], T[n], U[n,k] )
        end
        
        # 2.b) Corrección secuencial
        for n = 1:N_intervalos
            U[n+1, k+1] = F_sol[n] + G( T[n+1], T[n], U[n,k+1] ) 
            - G( T[n+1], T[n], U[n,k] )
        end
    end
    
    return U
    
end

EDP_secuencial (generic function with 1 method)

In [30]:
@time U1 = EDP_secuencial();

 40.763209 seconds (8.54 M allocations: 9.584 GiB, 2.29% gc time)


In [31]:
for i=1:N_intervalos+1
    sol = U1[i,end]
    writevtk(Ω,"Sol_numerica$(lpad(i,2,'0'))",cellfields=["sol" => sol])
end 


In [16]:
for i=1:N_intervalos+1
    int = ∫(U1[i,end])*dΩ
    println(Base.sum(int))
end 

0.003262844885942757
0.003262844921863485
0.003262845021583042
0.0032628452905109307
0.003262845964821981
0.0032628474919287834
0.0032628506018971717
0.0032628563509431705
0.003262866130102479
0.0032628816431595964
0.0032629048637482726
0.0032629379820114913
0.0032629833483535033
0.0032630434180281383
0.003263120697286006
0.0032632176902511107
0.003263336845517065
0.0032634805021409703
0.0032636508357056825
0.00326384980600489
0.003264079108475294
0.0032643401317249045
0.0032646339234446704
0.0032649611667247896
0.003265322168382636
0.0032657168603702586
0.0032661448146671435
0.003266605271283957
0.003267097178131688
0.0032676192406100313
0.0032681699779288638
0.003268747782504441
0.0032693509783687583
0.0032699778744749953
0.003270626809108839
0.003271296182312795
0.003271984474236769
0.0032726902485418342
0.0032734121412794797
0.0032741488369114695
0.003274899034202764
0.0032756614055112383
0.003276434553449215
0.003277216968977113
0.003278006994722232
0.003278802796742185
0.00327960